# Export YOLOv11 to ONNX and Upload to S3/MinIO

This notebook demonstrates:
- Exporting YOLOv11 model to ONNX format
- Uploading the ONNX model to S3/MinIO storage
- Using environment variables for configuration

## Prerequisites
Set the following environment variables before running:
- `AWS_ACCESS_KEY_ID`: S3 access key
- `AWS_SECRET_ACCESS_KEY`: S3 secret key
- `AWS_S3_ENDPOINT`: S3 endpoint URL
- `AWS_S3_BUCKET`: S3 bucket name

In [ ]:
import sys
sys.path.insert(0, '..')

import os
from pathlib import Path
from ultralytics import YOLO
import boto3
from botocore.exceptions import ClientError

print("Libraries imported successfully")

## 1. Configuration from Environment Variables

In [ ]:
# Get configuration from environment variables
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_S3_ENDPOINT = os.getenv('AWS_S3_ENDPOINT')
AWS_S3_BUCKET = os.getenv('AWS_S3_BUCKET', 'models')  # Default to 'models'
AWS_DEFAULT_REGION = os.getenv('AWS_DEFAULT_REGION', 'us-east-1')

# Validate configuration
print("Configuration:")
print(f"  AWS_ACCESS_KEY_ID: {'✓ Set' if AWS_ACCESS_KEY_ID else '✗ Not set'}")
print(f"  AWS_SECRET_ACCESS_KEY: {'✓ Set' if AWS_SECRET_ACCESS_KEY else '✗ Not set'}")
print(f"  AWS_S3_ENDPOINT: {AWS_S3_ENDPOINT or '✗ Not set'}")
print(f"  AWS_S3_BUCKET: {AWS_S3_BUCKET}")
print(f"  AWS_DEFAULT_REGION: {AWS_DEFAULT_REGION}")

if not all([AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_S3_ENDPOINT]):
    print("\n⚠️  Warning: Some environment variables are not set!")
    print("Please set AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, and AWS_S3_ENDPOINT")
else:
    print("\n✓ All required environment variables are set")

## 2. Download YOLOv11 Model (if not present)

In [ ]:
# Define paths
MODELS_DIR = Path('../models')
MODELS_DIR.mkdir(exist_ok=True, parents=True)

MODEL_PT_PATH = MODELS_DIR / 'yolo11n.pt'
MODEL_ONNX_PATH = MODELS_DIR / 'yolo11n.onnx'

print(f"Models directory: {MODELS_DIR.absolute()}")
print(f"PyTorch model path: {MODEL_PT_PATH}")
print(f"ONNX model path: {MODEL_ONNX_PATH}")

# Download model if not present
if not MODEL_PT_PATH.exists():
    print("\nDownloading YOLOv11n model...")
    
    # Change to models directory to download there (avoids permission issues)
    original_dir = os.getcwd()
    os.chdir(MODELS_DIR)
    
    try:
        model = YOLO('yolo11n.pt')
        print(f"✓ Model downloaded to {MODEL_PT_PATH}")
    finally:
        # Return to original directory
        os.chdir(original_dir)
else:
    print(f"\n✓ Model already exists at {MODEL_PT_PATH}")
    print(f"  Size: {MODEL_PT_PATH.stat().st_size / (1024**2):.2f} MB")

## 3. Export Model to ONNX Format

In [ ]:
# Export parameters
EXPORT_PARAMS = {
    'format': 'onnx',
    'imgsz': 640,
    'dynamic': True,
    'simplify': True,
    'opset': 17
}

print("Export configuration:")
for key, value in EXPORT_PARAMS.items():
    print(f"  {key}: {value}")

# Load model
print(f"\nLoading model from {MODEL_PT_PATH}...")
model = YOLO(str(MODEL_PT_PATH))

# Export to ONNX
print("\nExporting to ONNX format...")
print("This may take a few minutes...\n")

exported_path = model.export(**EXPORT_PARAMS)

print(f"\n✓ Export successful!")
print(f"  Exported to: {exported_path}")

In [ ]:
# Move ONNX file to models directory if needed
import shutil

exported_path = Path(exported_path)
if exported_path != MODEL_ONNX_PATH:
    print(f"Moving {exported_path} to {MODEL_ONNX_PATH}...")
    if MODEL_ONNX_PATH.exists():
        MODEL_ONNX_PATH.unlink()
    shutil.move(str(exported_path), str(MODEL_ONNX_PATH))
    print("✓ Model moved")

# Display model info
model_size_mb = MODEL_ONNX_PATH.stat().st_size / (1024**2)
print(f"\nONNX Model Information:")
print(f"  Path: {MODEL_ONNX_PATH}")
print(f"  Size: {model_size_mb:.2f} MB")
print(f"  Format: ONNX")
print(f"  Input size: 640x640")
print(f"  Dynamic shapes: Enabled")

## 4. Upload Model to S3/MinIO

In [ ]:
# Initialize S3 client
print("Initializing S3 client...")
print(f"  Endpoint: {AWS_S3_ENDPOINT}")
print(f"  Region: {AWS_DEFAULT_REGION}")

s3_client = boto3.client(
    's3',
    endpoint_url=AWS_S3_ENDPOINT,
    region_name=AWS_DEFAULT_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

print("✓ S3 client initialized")

In [ ]:
# Check if bucket exists
print(f"\nChecking bucket '{AWS_S3_BUCKET}'...")

try:
    s3_client.head_bucket(Bucket=AWS_S3_BUCKET)
    print(f"✓ Bucket '{AWS_S3_BUCKET}' exists")
except ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code == '404':
        print(f"Bucket '{AWS_S3_BUCKET}' not found, creating...")
        try:
            s3_client.create_bucket(Bucket=AWS_S3_BUCKET)
            print(f"✓ Bucket '{AWS_S3_BUCKET}' created")
        except ClientError as create_error:
            print(f"✗ Failed to create bucket: {create_error}")
            raise
    else:
        print(f"✗ Error checking bucket: {e}")
        raise

In [ ]:
# Upload model
object_key = MODEL_ONNX_PATH.name
s3_uri = f"s3://{AWS_S3_BUCKET}/{object_key}"

print(f"\nUploading model to S3...")
print(f"  Source: {MODEL_ONNX_PATH}")
print(f"  Destination: {s3_uri}")
print(f"  Size: {model_size_mb:.2f} MB")
print("\nUploading...")

try:
    # Upload file (using upload_file for better MinIO compatibility)
    s3_client.upload_file(
        str(MODEL_ONNX_PATH),
        AWS_S3_BUCKET,
        object_key
    )
    
    print("\n✓ Upload successful!")
    print(f"\nModel location:")
    print(f"  S3 URI: {s3_uri}")
    print(f"  Endpoint: {AWS_S3_ENDPOINT}")
    print(f"  Bucket: {AWS_S3_BUCKET}")
    print(f"  Key: {object_key}")
    
except ClientError as e:
    print(f"\n✗ Upload failed: {e}")
    raise

## 5. Verify Upload

In [ ]:
# Verify the object exists in S3
print("Verifying upload...\n")

try:
    response = s3_client.head_object(Bucket=AWS_S3_BUCKET, Key=object_key)
    
    print("✓ Object verified in S3")
    print(f"\nObject metadata:")
    print(f"  Content-Length: {response['ContentLength'] / (1024**2):.2f} MB")
    print(f"  Content-Type: {response.get('ContentType', 'N/A')}")
    print(f"  ETag: {response.get('ETag', 'N/A')}")
    print(f"  Last-Modified: {response.get('LastModified', 'N/A')}")
    
except ClientError as e:
    print(f"✗ Verification failed: {e}")
    raise

## 6. List All Objects in Bucket

In [ ]:
# List all objects in the bucket
print(f"Objects in bucket '{AWS_S3_BUCKET}':\n")

try:
    response = s3_client.list_objects_v2(Bucket=AWS_S3_BUCKET)
    
    if 'Contents' in response:
        for obj in response['Contents']:
            size_mb = obj['Size'] / (1024**2)
            print(f"  📄 {obj['Key']}")
            print(f"     Size: {size_mb:.2f} MB")
            print(f"     Last Modified: {obj['LastModified']}")
            print()
    else:
        print("  (empty bucket)")
        
except ClientError as e:
    print(f"✗ Failed to list objects: {e}")

## 7. Generate InferenceService YAML

Generate the YAML configuration for deploying on OpenShift AI.

In [ ]:
# Generate InferenceService YAML
inference_service_yaml = f"""apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: yolo11-person-detection
  namespace: train-detection
  labels:
    app: train-detection
    model: yolo11
  annotations:
    serving.kserve.io/deploymentMode: RawDeployment
    serving.knative.openshift.io/enablePassthrough: "true"
spec:
  predictor:
    model:
      modelFormat:
        name: onnx
        version: "1"
      runtime: yolo11-triton-runtime
      storageUri: {s3_uri}
      resources:
        requests:
          memory: 2Gi
          cpu: 1
        limits:
          memory: 4Gi
          cpu: 2
    minReplicas: 1
    maxReplicas: 3
"""

print("Generated InferenceService YAML:")
print("=" * 70)
print(inference_service_yaml)
print("=" * 70)

# Save to file
yaml_path = Path('../openshift/model-serving/inference-service-generated.yaml')
with open(yaml_path, 'w') as f:
    f.write(inference_service_yaml)

print(f"\n✓ YAML saved to: {yaml_path}")
print("\nTo deploy, run:")
print(f"  oc apply -f {yaml_path}")

## Summary

This notebook:
1. ✓ Loaded configuration from environment variables
2. ✓ Downloaded YOLOv11 model (if needed)
3. ✓ Exported model to ONNX format
4. ✓ Uploaded ONNX model to S3/MinIO
5. ✓ Verified the upload
6. ✓ Generated InferenceService YAML

### Next Steps

1. **Deploy ServingRuntime** (if not already deployed):
   ```bash
   oc apply -f ../openshift/model-serving/serving-runtime.yaml
   ```

2. **Deploy InferenceService**:
   ```bash
   oc apply -f ../openshift/model-serving/inference-service-generated.yaml
   ```

3. **Check deployment status**:
   ```bash
   oc get inferenceservice yolo11-person-detection -w
   ```

4. **Test the API** using `notebooks/03_api_detection.ipynb`